**Note**: At this time, the PaLM API is [only available in certain regions](https://developers.generativeai.google/available_regions).

## Setup Environment

In [ ]:
!pip install google-generativeai

In [ ]:
import google.generativeai as palm
import os

To get started, you'll need to [create an API key](https://developers.generativeai.google/tutorials/setup).

In [ ]:
#@title Insert PaLM API Key here
palm.configure(api_key='INSERT_KEY_HERE')

In [ ]:
# Use the palm.list_models function to find available models
# PaLM 2 available in 4 sizes: Gecko, Otter, Bison and Unicorn (largest)
# https://developers.generativeai.google/models/language

models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

## Simplest Chatbot

In [ ]:
# Set your input text
prompt = "Why is the sky blue?"
# prompt = "What is Quantum Computing? Explain like I'm 5."

completion = palm.generate_text(
    model=model,
    prompt=prompt,
    temperature=0,
    # temperature=0 >> more deterministic results // temperature=1 >> more randomness
    max_output_tokens=100
    # maximum length of response
)

print(completion.result)

## Custom AI Applications

### Code Explainer

In [ ]:
def get_completion(prompt):
  completion = palm.generate_text(
      model=model,
      prompt=prompt,
      temperature=0,
      # The maximum length of the response
      max_output_tokens=500,
      )
  response = completion.result
  return response

#### Simple Prompting

In [ ]:
# input_code = f"""
# x = [1, 2, 3, 4, 5]
# y = [i**2 for i in x if i%2 == 0]
# print(y)
# """

input_code = f"""
def my_func(x):
    if x > 5:
        return "High"
    else:
        return "Low"
result = my_func(4) + my_func(6) + my_func(4)
print(result)
"""

In [ ]:
prompt =f"""
Your task is to act as a Python Code Explainer.
I'll give you a Code Snippet.
Your job is to explain the Code Snippet step-by-step.
Also, compute the final output of the code.
Code Snippet is shared below, delimited with triple backticks:
```
{input_code}
```
"""

print(prompt)

In [ ]:
print(get_completion(prompt))

#### Effective Prompting

In [ ]:
python_code_examples = f"""
---------------------
Example 1: Code Snippet
x = 10
def foo():
    global x
    x = 5
foo()
print(x)

Correct answer: 5
Explanation: Inside the foo function, the global keyword is used to modify the global variable x to be 5.
So, print(x) outside the function prints the modified value, which is 5.
---------------------
Example 2: Code Snippet
def modify_list(input_list):
    input_list.append(4)
    input_list = [1, 2, 3]
my_list = [0]
modify_list(my_list)
print(my_list)

Correct answer: [0, 4]
Explanation: Inside the modify_list function, an element 4 is appended to input_list.
Then, input_list is reassigned to a new list [1, 2, 3], but this change doesn't affect the original list.
So, print(my_list) outputs [0, 4].
---------------------
"""

In [ ]:
# language = "Python"

# input_code = f"""
# def my_func(x):
#     if x > 5:
#         return "High"
#     else:
#         return "Low"
# result = my_func(4) + my_func(6) + my_func(4)
# print(result)
# """

input_code = f"""
def func(x):
    if x > 0:
        return x + func(x-1)
    return 0

result = func(5)
print(result)
"""

In [ ]:
prompt = f"""
Your task is to act as a Python Code Explainer.
I'll give you a Code Snippet.
Your job is to explain the Code Snippet step-by-step.
Break down the code into as many steps as possible.
Share intermediate checkpoints along with results.
State your Steps and Checkpoints in your output.
Few good examples of Python code output between #### separator:
####
{python_code_examples}
####
Code Snippet is shared below, delimited with triple backticks:
```
{input_code}
```
"""

print(prompt)

In [ ]:
print(get_completion(prompt))

### Code Explainer Tool App

In [ ]:
!pip install gradio

In [ ]:
#@title Gradio App
import gradio as gr
import os
import google.generativeai as palm

# load model
# PaLM API Key here
palm.configure(api_key='INSERT_KEY_HERE')


# define completion function
def get_completion(code_snippet):

  python_code_examples = f"""
  ---------------------
  Example 1: Code Snippet
  x = 10
  def foo():
      global x
      x = 5
  foo()
  print(x)
  Correct output: 5
  Code Explanation: Inside the foo function, the global keyword is used to modify the global variable x to be 5.
  So, print(x) outside the function prints the modified value, which is 5.
  ---------------------
  Example 2: Code Snippet
  def modify_list(input_list):
      input_list.append(4)
      input_list = [1, 2, 3]
  my_list = [0]
  modify_list(my_list)
  print(my_list)
  Correct output: [0, 4]
  Code Explanation: Inside the modify_list function, an element 4 is appended to input_list.
  Then, input_list is reassigned to a new list [1, 2, 3], but this change doesn't affect the original list.
  So, print(my_list) outputs [0, 4].
  ---------------------
  """

  prompt = f"""
  Your task is to act as a Python Code Explainer.
  I'll give you a Code Snippet.
  Your job is to explain the Code Snippet step-by-step.
  Break down the code into as many steps as possible.
  Share intermediate checkpoints & steps along with results.
  Few good examples of Python code output between #### separator:
  ####
  {python_code_examples}
  ####
  Code Snippet is shared below, delimited with triple backticks:
  ```
  {code_snippet}
  ```
  """

  completion = palm.generate_text(
      model=model,
      prompt=prompt,
      temperature=0,
      # The maximum length of the response
      max_output_tokens=500,
      )
  response = completion.result
  return response

# define app UI
iface = gr.Interface(fn=get_completion, inputs=[gr.Textbox(label="Insert Code Snippet",lines=5)],
                    outputs=[gr.Textbox(label="Explanation Here",lines=5)],
                    title="Python Code Explainer"
                    )

iface.launch(share=True)

In [ ]:
iface.close()

### AI Tool Ideas

#### Text Summarizer

In [ ]:
!pip install pypdf

In [ ]:
from pypdf import PdfReader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
directory = '/content/drive/MyDrive/1-Build-a-Chatbot/'
filename  = 'Attention-Is-All-You-Need.pdf'
# filename  = 'Art-of-War.pdf'

In [ ]:
# creating a pdf file object
pdfFileObject = open(directory+filename, 'rb')
# creating a pdf reader object
pdfReader = PdfReader(pdfFileObject)
text=[]
summary=' '
#Storing the pages in a list
for i in range(0,len(pdfReader.pages)):
  # creating a page object
  pageObj = pdfReader.pages[i].extract_text()
  pageObj= pageObj.replace('\t\r','')
  pageObj= pageObj.replace('\xa0','')
  # extracting text from page
  text.append(pageObj)

In [ ]:
# Merge multiple page - to reduce API Calls
def join_elements(lst, chars_per_element):
    new_lst = []
    for i in range(0, len(lst), chars_per_element):
        new_lst.append(''.join(lst[i:i+chars_per_element]))
    return new_lst

# Option to keep x elements per list element
new_text = join_elements(text, 3)

print(f"Original Pages = ",len(text))
print(f"Compressed Pages = ",len(new_text))

In [ ]:
def get_completion(prompt):
  completion = palm.generate_text(model=model,
                                  prompt=prompt,
                                  temperature=0,
                                  # The maximum length of the response
                                  max_output_tokens=200,
                                  )
  return completion.result

In [ ]:
summary = ""
for i in range(len(new_text)):
  prompt =f"""
  Your task is to act as a Text Summariser.
  I'll give you text from  pages of a book from beginning to end.
  And your job is to summarise text from these pages in less than 100 words.
  Don't be conversational. I need a plain 100 word answer.
  Text is shared below, delimited with triple backticks:
  ```{text[i]}```
  """
  try:
    response = get_completion(prompt)
  except:
    response = get_completion(prompt)
  print(response)
  summary= f"{summary} {response}\n\n"
  # result.append(response)
  time.sleep(19)  #You can query the model only 3 times in a minute for free, so we need to put some delay

In [ ]:
with open(directory +'/palm_api_summary.txt',
          'w') as out:
  out.write(summary)

#### Mock Interview

In [ ]:
# Build a Chatbot that takes a Mock Interview
# For a specific Job Role, provided as input by End-User.

#### AI Teacher

In [ ]:
# This AI Teacher explains you topics in ELI-X level
# Where "X" is the age-appropriate learner level.

#### Code Reviewer

In [ ]:
# AI helper that generates comprehensive review that cover code
# Clarity, structure, efficiency, adherence
# To best practices, and maintainability.